In [25]:
import os
import numpy as np
import tensorflow as tf
import scipy.io
import cv2
from tensorflow.keras import layers, Model
from matplotlib import pyplot as plt

In [26]:
TRAIN_IMG_DIR = "/home/nikola/Documents/Assignment-8/data/part_A/train_data/images"
TRAIN_GT_DIR = "/home/nikola/Documents/Assignment-8/data/part_A/train_data/ground_truth"
TEST_IMG_DIR  = "/home/nikola/Documents/Assignment-8/data/part_A/test_data/images"
TEST_GT_DIR   = "/home/nikola/Documents/Assignment-8/data/part_A/test_data/ground_truth"

In [27]:
def load_density_map(mat_path):
    mat = scipy.io.loadmat(mat_path)
    density_map = mat["density"]
    return density_map

def load_image_and_density(image_path, gt_path):
    # Convert EagerTensors to Python strings
    image_path = image_path.numpy().decode("utf-8")
    gt_path = gt_path.numpy().decode("utf-8")

    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (256, 256)) / 255.0

    density = load_density_map(gt_path)
    density = cv2.resize(density, (64, 64))
    density = np.expand_dims(density, axis=-1)

    return image.astype(np.float32), density.astype(np.float32)

def tf_load_image_and_density(image_path, gt_path):
    image, density = tf.py_function(load_image_and_density, [image_path, gt_path], [tf.float32, tf.float32])
    image.set_shape((256, 256, 3))
    density.set_shape((64, 64, 1))
    return image, density


In [28]:
train_img_paths = sorted([os.path.join(TRAIN_IMG_DIR, f) for f in os.listdir(TRAIN_IMG_DIR) if f.endswith(".jpg")])
train_gt_paths  = [p.replace("images", "ground_truth").replace(".jpg", ".mat") for p in train_img_paths]

test_img_paths = sorted([os.path.join(TEST_IMG_DIR, f) for f in os.listdir(TEST_IMG_DIR) if f.endswith(".jpg")])
test_gt_paths  = [p.replace("images", "ground_truth").replace(".jpg", ".mat") for p in test_img_paths]

BATCH_SIZE = 8

train_ds = tf.data.Dataset.from_tensor_slices((train_img_paths, train_gt_paths))
train_ds = train_ds.map(tf_load_image_and_density, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(100).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_img_paths, test_gt_paths))
test_ds = test_ds.map(tf_load_image_and_density, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [29]:
def unet_model(output_channels=1):
    inputs = tf.keras.layers.Input(shape=(256, 256, 3))

    # Encoder
    x1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    x1 = layers.Conv2D(64, 3, activation='relu', padding='same')(x1)
    p1 = layers.MaxPooling2D((2,2))(x1)

    x2 = layers.Conv2D(128, 3, activation='relu', padding='same')(p1)
    x2 = layers.Conv2D(128, 3, activation='relu', padding='same')(x2)
    p2 = layers.MaxPooling2D((2,2))(x2)

    x3 = layers.Conv2D(256, 3, activation='relu', padding='same')(p2)
    x3 = layers.Conv2D(256, 3, activation='relu', padding='same')(x3)
    p3 = layers.MaxPooling2D((2,2))(x3)

    # Bottleneck
    b = layers.Conv2D(512, 3, activation='relu', padding='same')(p3)
    b = layers.Conv2D(512, 3, activation='relu', padding='same')(b)

    # Decoder
    u1 = layers.Conv2DTranspose(256, 2, strides=2, padding='same')(b)
    u1 = layers.concatenate([u1, x3])
    u1 = layers.Conv2D(256, 3, activation='relu', padding='same')(u1)
    u1 = layers.Conv2D(256, 3, activation='relu', padding='same')(u1)

    u2 = layers.Conv2DTranspose(128, 2, strides=2, padding='same')(u1)
    u2 = layers.concatenate([u2, x2])
    u2 = layers.Conv2D(128, 3, activation='relu', padding='same')(u2)
    u2 = layers.Conv2D(128, 3, activation='relu', padding='same')(u2)

    u3 = layers.Conv2DTranspose(64, 2, strides=2, padding='same')(u2)
    u3 = layers.concatenate([u3, x1])
    u3 = layers.Conv2D(64, 3, activation='relu', padding='same')(u3)
    u3 = layers.Conv2D(64, 3, activation='relu', padding='same')(u3)

    outputs = layers.Conv2D(output_channels, 1, activation='linear')(u3)
    return Model(inputs, outputs)


In [30]:
for img_batch, gt_batch in train_ds.take(1):
    print("Image batch shape:", img_batch.shape, img_batch.dtype)
    print("GT batch shape:", gt_batch.shape, gt_batch.dtype)


2025-07-08 00:08:30.727042: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: FileNotFoundError: [Errno 2] No such file or directory: '/home/nikola/Documents/Assignment-8/data/part_A/train_data/ground_truth/IMG_10.mat'
Traceback (most recent call last):

  File "/home/nikola/.local/lib/python3.11/site-packages/scipy/io/matlab/_mio.py", line 39, in _open_file
    return open(file_like, mode), True
           ^^^^^^^^^^^^^^^^^^^^^

FileNotFoundError: [Errno 2] No such file or directory: '/home/nikola/Documents/Assignment-8/data/part_A/train_data/ground_truth/IMG_10.mat'


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/home/nikola/.local/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/nikola/.local/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 146, in __call__
  

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} FileNotFoundError: [Errno 2] No such file or directory: '/home/nikola/Documents/Assignment-8/data/part_A/train_data/ground_truth/IMG_1.mat'
Traceback (most recent call last):

  File "/home/nikola/.local/lib/python3.11/site-packages/scipy/io/matlab/_mio.py", line 39, in _open_file
    return open(file_like, mode), True
           ^^^^^^^^^^^^^^^^^^^^^

FileNotFoundError: [Errno 2] No such file or directory: '/home/nikola/Documents/Assignment-8/data/part_A/train_data/ground_truth/IMG_1.mat'


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/home/nikola/.local/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/nikola/.local/lib/python3.11/site-packages/tensorflow/python/ops/

In [ ]:
model = unet_model(output_channels=1)
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=MeanSquaredError(),
              metrics=['mae'])

model.fit(train_ds,
          validation_data=test_ds,
          epochs=10)


Epoch 1/10


ValueError: in user code:

    File "/home/nikola/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/nikola/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/nikola/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/nikola/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1151, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/nikola/.local/lib/python3.11/site-packages/keras/src/engine/training.py", line 1209, in compute_loss
        return self.compiled_loss(
    File "/home/nikola/.local/lib/python3.11/site-packages/keras/src/engine/compile_utils.py", line 277, in __call__
        

In [ ]:
def plot_density_map(image, true_density, pred_density):
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1)
    plt.imshow(image)
    plt.title("Input Image")
    plt.axis('off')

    plt.subplot(1,3,2)
    plt.imshow(true_density[:,:,0], cmap='jet')
    plt.title(f"Ground Truth Count: {tf.reduce_sum(true_density):.1f}")
    plt.axis('off')

    plt.subplot(1,3,3)
    plt.imshow(pred_density[:,:,0], cmap='jet')
    plt.title(f"Predicted Count: {tf.reduce_sum(pred_density):.1f}")
    plt.axis('off')
    plt.show()